## Introduction
- We will analysis Toronto Neighborhoods to select best locations to open a new coffee store.
- First, we will load the data.
- Then, we will add latitude and logitude to it
- Then, we will get top most venues for them
- Then, we will select neghborhoods having coffeeshop as thier main venues category, where we will open our new store.

### Import libraries

In [30]:
from bs4 import BeautifulSoup
import requests
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


### Read the wikipedia page into a dataframe and remove not assigned

In [31]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url)
soup = BeautifulSoup(data.text)
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [32]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


### Get coords from CSV File

In [33]:
#read coords from csv file (file name changed)
coords_df = pd.read_csv('coords.csv')
coords_df.columns = df.columns
coords_df.head()

,PostalCode,Borough,Neighborhood
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Combine coords with our data

In [34]:
df = df.merge(coords_df,on='PostalCode',how='inner')
df.head()

,PostalCode,Borough_x,Neighborhood_x,Borough_y,Neighborhood_y
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [35]:
df.columns = ['PostalCode','Borough','Neighborhood','Latitude','Longitude']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


### Toronto Exploration 

### Create a map of Tornoto with Neighborhoods on it

In [36]:
address = 'Toronto, Ontario, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornto City are 43.6534817, -79.3839347.


In [37]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define foursquare client and version

In [38]:
CLIENT_ID = 'BNZP4HMPDS2JN0ETSYEEWSO4C2MM0G1TH5LZJ1OTHFUYN3WS' # your Foursquare ID
CLIENT_SECRET = '3P0HIAXHLRPO3O0VN4EG1ZR0VDPOXXN35ROVXUUIUKFR3NJ3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BNZP4HMPDS2JN0ETSYEEWSO4C2MM0G1TH5LZJ1OTHFUYN3WS
CLIENT_SECRET:3P0HIAXHLRPO3O0VN4EG1ZR0VDPOXXN35ROVXUUIUKFR3NJ3


#### Explore neighborhoods in Toronto

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  );

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

#### Let's find out how many unique categories can be curated from all the returned venues

In [41]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 274 uniques categories.


## 3. Analyze Each Neighborhood

In [42]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1  Latin American Restaurant  0.25
2             Breakfast Spot  0.25
3               Skating Rink  0.25
4              Metro Station  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.25
1  Skating Rink  0.12
2    Playground  0.12
3           Pub  0.12
4   Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.10
1     Coffee Shop  0.10
2      Restaurant  0.05
3  Ice Cream Shop  0.05
4     Supermarket  0.05


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Café  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4  Moroccan Restaurant  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.09
1         Coffee Shop  0.09
2      Sandwich Place  0.09
3             Butcher  0.05
4     Thai Restaurant

#### Let's put that into a _pandas_ dataframe


In [44]:
#First, let's write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Latin American Restaurant,Skating Rink,Breakfast Spot,Drugstore,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
1,"Alderwood, Long Branch",Pizza Place,Coffee Shop,Gym,Skating Rink,Sandwich Place,Playground,Pub,Discount Store,Department Store,Dessert Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Fried Chicken Joint,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Mobile Phone Shop,Sushi Restaurant
3,Bayview Village,Café,Bank,Japanese Restaurant,Chinese Restaurant,Dim Sum Restaurant,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Pizza Place,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant,Restaurant,Café


### Get Neighborhoods woth "Coffee Shop" as their 1st common venues

In [48]:
coffee_shop_venues = neighborhoods_venues_sorted[neighborhoods_venues_sorted['1st Most Common Venue'] == 'Coffee Shop']
coffee_shop_venues.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Gas Station,Fried Chicken Joint,Ice Cream Shop,Diner,Deli / Bodega,Middle Eastern Restaurant,Mobile Phone Shop,Sushi Restaurant
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Pizza Place,Juice Bar,Fast Food Restaurant,Butcher,Indian Restaurant,Restaurant,Café
5,Berczy Park,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Seafood Restaurant,Cheese Shop,Pharmacy,Farmers Market,Beer Bar,Bistro
11,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Salad Place,Bubble Tea Shop,Burger Joint,Modern European Restaurant,Juice Bar,Korean Restaurant
17,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Hotel,Café,Italian Restaurant,Gym,American Restaurant,Japanese Restaurant,Deli / Bodega,Seafood Restaurant


### Get list of prospective Neighborhoods to open the new coffee  store

In [51]:
df_final = coffee_shop_venues[['Neighborhood']]
df_final

,Neighborhood
2,"Bathurst Manor, Wilson Heights, Downsview North"
4,"Bedford Park, Lawrence Manor East"
5,Berczy Park
11,Central Bay Street
17,"Commerce Court, Victoria Hotel"
29,Enclave of L4W
31,Enclave of M5E
34,"First Canadian Place, Underground city"
36,"Garden District, Ryerson"
40,"Harbourfront East, Union Station, Toronto Islands"


In [55]:
df_final = df_final.merge(df,on='Neighborhood')
df_final

,Neighborhood,PostalCode,Borough,Latitude,Longitude
0,"Bathurst Manor, Wilson Heights, Downsview North",M3H,North York,43.754328,-79.442259
1,"Bedford Park, Lawrence Manor East",M5M,North York,43.733283,-79.419750
2,Berczy Park,M5E,Downtown Toronto,43.644771,-79.373306
3,Central Bay Street,M5G,Downtown Toronto,43.657952,-79.387383
4,"Commerce Court, Victoria Hotel",M5L,Downtown Toronto,43.648198,-79.379817
5,Enclave of L4W,M7R,Mississauga,43.636966,-79.615819
6,Enclave of M5E,M5W,Downtown Toronto Stn A,43.646435,-79.374846
7,"First Canadian Place, Underground city",M5X,Downtown Toronto,43.648429,-79.382280
8,"Garden District, Ryerson",M5B,Downtown Toronto,43.657162,-79.378937
9,"Harbourfront East, Union Station, Toronto Islands",M5J,Downtown Toronto,43.640816,-79.381752


### Group them by Borough and sort them

In [61]:
df_final_grouped = df_final[['Borough','Neighborhood']].groupby(['Borough']).agg('count')

In [67]:
df_final_grouped.sort_values(by=['Neighborhood'],ascending=False)

,Neighborhood
Borough,
Downtown Toronto,11
Central Toronto,2
North York,2
Downtown Toronto Stn A,1
East Toronto,1
East York,1
Mississauga,1
Queen's Park,1
Scarborough,1


### Conclusion

The best borough to open a new coffee store is "Downtown Toronto" as it has 11 neighborhoods with coffee shop as thier main common venues